In [2]:
import pandas as pd
import numpy as np
import wbdata

In [3]:
def wb_clear(df, x):
    df.index = df.index.set_levels(df.index.levels[1].astype(int), level=1)
    df.index = df.index.set_levels(df.index.levels[0].str.lower(), level=0)
    df = df[df.index.get_level_values(1) >= 2006]
    df = df[df.loc[:, x].notna()]
    df.index = df.index.set_names('year', level=1)
    return df

In [4]:
def add_wb_var(code, var_name):
    x = wb_clear(wbdata.get_dataframe({code:var_name}), var_name)
    all_df = pd.read_csv('raw_data/master.csv').set_index(['country', 'year'])
    all_df = all_df.join(x).reset_index()
    all_df = all_df[(all_df.year == 2008) | (all_df.year == 2018)]

    m_row = [all_df.loc[i].isna().mean() > .15 for i in all_df.index]
    all_df = all_df.drop(np.array(all_df.index)[m_row], axis=0)

    m_col = [all_df.loc[:,i].isna().mean() > .15 for i in all_df.columns]
    all_df = all_df.drop(np.array(all_df.columns)[m_col], axis=1)

    no_data_country = np.array(list(all_df.groupby('country')\
        .groups.keys()))[list(all_df.groupby('country').count().min(axis=1) != 2)]
    all_df = all_df[~all_df.country.isin(no_data_country)]

    print(f"{list(zip(all_df.columns, [all_df.loc[:,i].isna().sum() for i in all_df.columns]))}\n\n{all_df.shape}")

    return all_df

Lisää koodi ja muuttujanimi `add_wb_var()` funktioon muodossa 'XX.XX.XX.XX' ja 'oma_nimi_muuttujalle'

In [5]:
df = add_wb_var('SE.SEC.ENRR', 'enrollment')

[('country', 0), ('year', 0), ('PR', 0), ('CL', 0), ('Status', 0), ('DD regime', 0), ('DD category', 0), ('Monarchy', 0), ('Commonwealth', 0), ('Democracy', 0), ('Presidential', 0), ('President name', 0), ('President accesion', 0), ('President birthyear', 0), ('Interim phase (more than 2 Presidents/year=1)', 0), ('Female president (0: No; 1: Yes)', 0), ('Colony', 0), ('Communist', 0), ('Regime change lag', 0), ('spatial democracy', 0), ('countrycode', 0), ('No. of chambers in parliament', 0), ('proportional voting', 0), ('Election system', 0), ('No. of members in lower house', 0), ('New constitution', 0), ('fullsuffrage', 0), ('electoral', 0), ('spatial electoral', 0), ('alternation', 0), ('multiparty', 0), ('free and fair election', 0), ('parliamentary election year', 0), ('postponed election', 0), ('Anticorruption policy', 0), ('Civil rights', 0), ('Civil society participation', 0), ('Civil society traditions', 0), ('Cleavage conflict management', 0), ('Commitment to democratic insti

In [6]:
df.to_csv('raw_data/small_master.csv')